In [ ]:
!pip install torchinfo fairseq transformers huggingface-hub

In [3]:
import torch
import torch.nn as nn
from fairseq.models import FairseqEncoder, FairseqDecoder, FairseqEncoderDecoderModel, register_model, FairseqModel
from transformers.models.whisper.modeling_whisper import WhisperPositionalEmbedding, WhisperSdpaAttention
from transformers.activations import GELUActivation
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from fairseq.data import Dictionary
import torchaudio
from datasets import load_dataset
from transformers import AutoProcessor
from torchaudio.transforms import MelSpectrogram
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from typing import Optional, Tuple, Union
import logging
import os
import sys
from torch import nn, Tensor
from fairseq import utils
from typing import Dict
import math
import torch
from torchaudio.transforms import MelSpectrogram
import torch
from IPython.display import Audio
import soundfile as sf

from fairseq.models import (
    FairseqEncoderDecoderModel,
    register_model,
    register_model_architecture,
)

try:
    from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperConfig
    has_hf = True
except ImportError:
    has_hf = False

logger = logging.getLogger(__name__)

from fairseq.models import FairseqEncoder, FairseqDecoder

In [4]:

#костыль  
class DummyEncoder(FairseqEncoder):
    def forward(self, *args, **kwargs):
        return None  # или возвращайте данные, если требуется.

#костыль  
class DummyDecoder(FairseqDecoder):
    def forward(self, *args, **kwargs):
        return None

@register_model("whisper-turbo")
class HuggingFaceWhisperModel(FairseqEncoderDecoderModel):
    def __init__(self, args):
        dummy_encoder = DummyEncoder(None)
        dummy_decoder = DummyDecoder(None)
        super().__init__(dummy_encoder, dummy_decoder)  # No encoder or decoder
        if not has_hf:
            raise ImportError(
                '\n\nPlease install huggingface/transformers with:'
                '\n\n  pip install transformers'
                '\n\nOr to make local edits, install the submodule:'
                '\n\n  git submodule update --init '
                'fairseq/models/huggingface/transformers'
            )
        self.args = args
        self.load_model(args)

    def load_model(self, args):
        model_path = getattr(args, 'load_hf_whisper_from', '')
        assert model_path, "Model path must be specified in --load-hf-whisper-from"
        self.model = WhisperForConditionalGeneration.from_pretrained(model_path)
        self.processor = AutoProcessor.from_pretrained(model_path)
        self.config = self.model.config

    @staticmethod
    def add_args(parser):
        parser.add_argument('--load-hf-whisper-from', type=str, default='',
                            help='load Hugging Face pretrained Whisper from path')
        parser.add_argument('--max-target-positions', type=int,
                            help='maximum target positions for the decoder')

    @classmethod
    def build_model(cls, args, task):
        default_architecture(args)
        return cls(args)

    def forward(
        self,
        src_tokens: Tensor,
        tgt_tokens: Optional[Tensor] = None,
        src_lengths: Optional[Tensor] = None,
        incremental_state: Optional[Dict[str, Dict[str, Optional[Tensor]]]] = None,
    ):
        input_features = src_tokens 
        
        if tgt_tokens is not None:
            outputs = self.model(
                input_features=input_features,
                decoder_input_ids=tgt_tokens,
            )
            logits = outputs.logits
        else:
            self.eval()
            with torch.no_grad():
                generated_ids = self.model.generate(
                    input_features=input_features
                )
            logits = generated_ids
    
        batch_size = input_features.size(0)
        seq_len = input_features.size(1)
        
        # Dummy values for hidden states and cells (as Whisper doesn't have these)
        # final_hiddens = torch.zeros(1, batch_size, self.config.d_model).to(input_features.device)
        # final_cells = torch.zeros(1, batch_size, self.config.d_model).to(input_features.device)
        # encoder_padding_mask = torch.zeros(seq_len, batch_size).to(input_features.device)
    
        return tuple(
            (
                logits,  # seq_len x batch x hidden (logits or other output)
                # final_hiddens,  # Dummy hidden states
                # final_cells,  # Dummy cell states
                # encoder_padding_mask,  # Dummy padding mask
                None,
                None,
                None
            )
        )

    def parse_waveform(self, file: str, **kwargs):
        waveform, sampling_rate = sf.read(file)
        waveform = torch.tensor(waveform).unsqueeze(0).float() 
        inputs = self.processor(waveform.squeeze(0), sampling_rate=sampling_rate, return_tensors="pt")
        waveform = inputs['input_features']
        
        return waveform



    def generate(self, audio_tokens=None, text=False, skip_special_tokens=True, file=None, **kwargs):
        if audio_tokens is None and file is None:
            raise Exception("audio_tokens or file must not be None")
        
        if file is not None:
            waveform = self.parse_waveform(file)
            audio_tokens = waveform
            
        self.eval()
        
        with torch.no_grad():
            generated_ids = self.model.generate(audio_tokens, **kwargs)

        if not text:
            return generated_ids

        return self.processor.batch_decode(generated_ids, skip_special_tokens)

def default_architecture(args):
    args.load_hf_whisper_from = getattr(args, 'load_hf_whisper_from', 'openai/whisper-large-v3-turbo')
    args.generate_text = getattr(args, 'generate_text', 'False')


In [8]:
class Args:
    pass

model = HuggingFaceWhisperModel.build_model(Args(), None)

In [9]:
# Загружаем тестовый датасет

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]


waveform = torch.tensor(sample['array']).unsqueeze(0)  # Add batch dimension
sampling_rate = sample['sampling_rate']

waveform = waveform.float()

inputs = model.processor(waveform.squeeze(0), sampling_rate=sampling_rate, return_tensors="pt")
waveform = inputs['input_features']

sf.write('audio.wav',sample['array'], sampling_rate)

In [10]:
# сгенерировать токены

model.generate(waveform)

tensor([[50258, 50259, 50360, 50364,  2221,    13,  2326,   388,   391,   307,
           264, 50244,   295,   264,  2808,  5359,    11,   293,   321,   366,
          5404,   281,  2928,   702, 14943,    13,  6966,   307,  2221,    13,
          2326,   388,   391,   311,  9060,  1570,  1880,   813,   702,  1871,
            13,   634,  5112,   505,   300,   412,   341, 42729,  3196,   295,
           264,  1064,    11,   365,  5272,   293, 12904,  9256,   450, 10539,
           949,   505,    11,  1034,  4680, 10117,   490,  3936,   293,  1080,
          3542,  5160,   881, 26336,   281,   264,  1575,    13,   634,   575,
         12525, 22618,  1968,  6144, 35617, 20084,  1756,   311,   589,   307,
           534, 10281,   934,   439,    11]])

In [11]:
# сгенерировать текст

model.generate(waveform, text=True)

[" Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Layton's work is really Greek after all,"]

In [12]:
# форвард пасс

in_features = model.processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features
prompt_ids = torch.tensor(model.processor.tokenizer.prefix_tokens).unsqueeze(0)
model(src_tokens=in_features, tgt_tokens=prompt_ids)

(tensor([[[ 2.4991,  1.7261, -0.9168,  ...,  0.1926,  2.3552, -0.5627],
          [-0.7742, -0.2981, -2.3134,  ..., -2.1932, -2.1076, -3.5427]]],
        grad_fn=<UnsafeViewBackward0>),
 None,
 None,
 None)

In [13]:
# сгенерировать из текст аудиофайла

model.generate(file='audio.wav', text=True)

[" Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Layton's work is really Greek after all,"]

In [14]:
# сгенерировать токены из аудиофайла

model.generate(file='audio.wav')

tensor([[50258, 50259, 50360, 50364,  2221,    13,  2326,   388,   391,   307,
           264, 50244,   295,   264,  2808,  5359,    11,   293,   321,   366,
          5404,   281,  2928,   702, 14943,    13,  6966,   307,  2221,    13,
          2326,   388,   391,   311,  9060,  1570,  1880,   813,   702,  1871,
            13,   634,  5112,   505,   300,   412,   341, 42729,  3196,   295,
           264,  1064,    11,   365,  5272,   293, 12904,  9256,   450, 10539,
           949,   505,    11,  1034,  4680, 10117,   490,  3936,   293,  1080,
          3542,  5160,   881, 26336,   281,   264,  1575,    13,   634,   575,
         12525, 22618,  1968,  6144, 35617, 20084,  1756,   311,   589,   307,
           534, 10281,   934,   439,    11]])